## Preprocessing

In [69]:
#  Import dependencies
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Read in the charity

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head(100)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
95,10845123,UNITED STATES BOWLING CONGRESS INC,T3,CompanySponsored,C1000,Preservation,Trust,1,0,N,5000,0
96,10845329,CIVITAN INTERNATIONAL,T3,CompanySponsored,C1000,Preservation,Association,1,0,N,5000,1
97,10845382,CALIFORNIA GARDEN CLUBS INC,T3,CompanySponsored,C1200,CommunityServ,Trust,1,0,N,5000,0
98,10846155,CALIFORNIA GARDEN CLUBS INC,T3,CompanySponsored,C1200,CommunityServ,Trust,1,0,N,5000,0


In [70]:
# # Dropping non-beneficial columns
application_df = application_df.drop(["EIN", "NAME", "STATUS"], axis=1)



In [71]:
# Binning for APPLICATION_TYPE
application_counts = application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = list(application_counts[application_counts < 1000].index)

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")


In [72]:
# Binning for CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < 5000].index)

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, 'Other')


In [73]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

# Splitting the data into features and target
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis=1).values

In [74]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Standardizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [81]:
# Defining the initial neural network model
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_104 (Dense)           (None, 80)                2960      
                                                                 
 dense_105 (Dense)           (None, 30)                2430      
                                                                 
 dense_106 (Dense)           (None, 1)                 31        
                                                                 
Total params: 5421 (21.18 KB)
Trainable params: 5421 (21.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [82]:
# Compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, verbose=1)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5790 - accuracy: 0.7182
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5654 - accuracy: 0.7256
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5633 - accuracy: 0.7275
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5621 - accuracy: 0.7286
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5605 - accuracy: 0.7299
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5598 - accuracy: 0.7295
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5592 - accuracy: 0.7303
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5582 - accuracy: 0.7299
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5579 - accuracy: 0.7293
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5573 - accura

In [83]:
# Evaluating the model
model_loss_attempt1, model_accuracy_attempt1 = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Attemt 1Loss: {model_loss_attempt1}, Accuracy: {model_accuracy_attempt1}")

268/268 - 0s - loss: 0.5649 - accuracy: 0.7224 - 349ms/epoch - 1ms/step
Attemt 1Loss: 0.5649368166923523, Accuracy: 0.722449004650116


## Attempt two:

In [94]:
# Attempt 2: Additional Hidden Layer and Dropout
# (Adding another hidden layer and a dropout layer)

# Defining the initial neural network model
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_115 (Dense)           (None, 80)                2960      
                                                                 
 dense_116 (Dense)           (None, 30)                2430      
                                                                 
 dense_117 (Dense)           (None, 20)                620       
                                                                 
 dense_118 (Dense)           (None, 10)                210       
                                                                 
 dense_119 (Dense)           (None, 1)                 11        
                                                                 
Total params: 6231 (24.34 KB)
Trainable params: 6231 (24.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [95]:
# Adding a dropout layer
nn.add(tf.keras.layers.Dropout(0.3))


In [96]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [97]:
# Train the model
fit_model_attempt2 = nn.fit(X_train_scaled, y_train, epochs=100, verbose=1)


Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 2.8323 - accuracy: 0.6434
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 2.8764 - accuracy: 0.6464
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 2.8863 - accuracy: 0.6474
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 2.9060 - accuracy: 0.6480
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 2.9403 - accuracy: 0.6448
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 2.8478 - accuracy: 0.6524
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 2.8280 - accuracy: 0.6513
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 2.8940 - accuracy: 0.6502
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 2.9038 - accuracy: 0.6481
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 2.8781 - accura

In [98]:
# Evaluate the model
model_loss_attempt2, model_accuracy_attempt2 = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Attempt 2 - Loss: {model_loss_attempt2}, Accuracy: {model_accuracy_attempt2}")

268/268 - 0s - loss: 0.6263 - accuracy: 0.6699 - 355ms/epoch - 1ms/step
Attempt 2 - Loss: 0.6262931823730469, Accuracy: 0.6698542237281799


## Attemp three

In [99]:
# Attempt 3: Adjusted Learning Rate and Batch Size
# (Modifying learning rate and batch size)

# Defining the initial neural network model
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_120 (Dense)           (None, 80)                2960      
                                                                 
 dense_121 (Dense)           (None, 30)                2430      
                                                                 
 dense_122 (Dense)           (None, 20)                620       
                                                                 
 dense_123 (Dense)           (None, 10)                210       
                                                                 
 dense_124 (Dense)           (None, 1)                 11        
                                                                 
Total params: 6231 (24.34 KB)
Trainable params: 6231 (24.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [100]:
# Modyfying learnig rate
# Compiling the model with a lower learning rate
nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])

In [101]:
# Trainig the model with smaller batch size
fit_model_attempt3 = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1)


Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6605 - accuracy: 0.6574
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5874 - accuracy: 0.7215
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5697 - accuracy: 0.7259
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5648 - accuracy: 0.7278
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5625 - accuracy: 0.7290
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5609 - accuracy: 0.7291
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5598 - accuracy: 0.7297
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5591 - accuracy: 0.7301
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7300
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5578 - accura

In [102]:
# Evaluating the model
model_loss_attempt3, model_accuracy_attempt3 = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Attempt 3 - Loss: {model_loss_attempt3}, Accuracy: {model_accuracy_attempt3}")

268/268 - 0s - loss: 0.5629 - accuracy: 0.7226 - 323ms/epoch - 1ms/step
Attempt 3 - Loss: 0.5628803968429565, Accuracy: 0.7225655913352966


In [103]:
# Saving the model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
